### Word Group Prepare For Youtube Tool

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = English
Lang Id = 1
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [6]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [7]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,and this is what it was for me and it was,2038,2040,and this is what it was for me and it was great,mO0oqUQ5eHc,https://www.youtube.com/watch?v=mO0oqUQ5eHc&t=...
1,what do we have to do and not do it this,2007,2010,but ok we dont what do we have to do and not d...,7eUrrQRpz2w,https://www.youtube.com/watch?v=7eUrrQRpz2w&t=...
2,and what this will do is this will,2226,2228,and what this will do is this will,UNnFRj9_TNo,https://www.youtube.com/watch?v=UNnFRj9_TNo&t=...
3,and what this will do is this will,274,282,and what this will do is this will,fgMD2wvpvpk,https://www.youtube.com/watch?v=fgMD2wvpvpk&t=...
4,what it is that you do and do not,624,626,what it is that you do and do not like,Gyqu5AER7gg,https://www.youtube.com/watch?v=Gyqu5AER7gg&t=...
...,...,...,...,...,...,...
371,it will,639,643,it will,LfnrRPFhkuY,https://www.youtube.com/watch?v=LfnrRPFhkuY&t=...
372,no will,557,558,no will,CkqI8IW8MlU,https://www.youtube.com/watch?v=CkqI8IW8MlU&t=...
373,it will,314,317,it will,zxiHkW22UrA,https://www.youtube.com/watch?v=zxiHkW22UrA&t=...
374,he will,2905,2907,he will,1Qz6fafTIEM,https://www.youtube.com/watch?v=1Qz6fafTIEM&t=...


In [8]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA,https://www.youtube.com/watch?v=IXhNNNeYXnA&t=...
1,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4,https://www.youtube.com/watch?v=_aBzOI9DWb4&t=...
2,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE,https://www.youtube.com/watch?v=ldlflaQVAAE&t=...
3,the way is where this is why i really want to ...,2610,2615,the way is where this is why i really want to ...,MXGmKbjEIlw,https://www.youtube.com/watch?v=MXGmKbjEIlw&t=...
4,say oh yeah yes she can and she would like to ...,1637,1644,say oh yeah yes she can and she would like to ...,dYa5scWadpc,https://www.youtube.com/watch?v=dYa5scWadpc&t=...
...,...,...,...,...,...,...
695,too much time before can help,10636,10638,too much time before can help,Yf7Vs6IWlLk,https://www.youtube.com/watch?v=Yf7Vs6IWlLk&t=...
696,really know who god was still,252,256,so i didnt really know who god was still,T_seNRh76aU,https://www.youtube.com/watch?v=T_seNRh76aU&t=...
697,anything going on or anything,1487,1490,to hear anything going on or anything,WqKV2veBRlc,https://www.youtube.com/watch?v=WqKV2veBRlc&t=...
698,never be good at anything now,127,129,ill never be good at anything now,WdnoMJ9KyX4,https://www.youtube.com/watch?v=WdnoMJ9KyX4&t=...


In [9]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,now this shot could have been the top shot of ...,199,206,now this shot could have been the top shot of ...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
1,so you can imagine if you go through all of th...,293,306,so you can imagine if you go through all of th...,K1GhQ8633Bg,https://www.youtube.com/watch?v=K1GhQ8633Bg&t=...
2,and so one of the questions people say to me i...,971,978,craig robinson and so one of the questions peo...,G3w3f6M_w54,https://www.youtube.com/watch?v=G3w3f6M_w54&t=...
3,its a little family that you turn into and jus...,49,59,its a little family that you turn into and jus...,GymR27z8RHM,https://www.youtube.com/watch?v=GymR27z8RHM&t=49s
4,and i remember when it happened i was like oh ...,796,803,ayanna howard and i remember when it happened ...,M_IRnJZ2xQ0,https://www.youtube.com/watch?v=M_IRnJZ2xQ0&t=...
...,...,...,...,...,...,...
695,because he was missing out on many fun things,453,456,because he was missing out on many fun things,L5jE2X3NZaM,https://www.youtube.com/watch?v=L5jE2X3NZaM&t=...
696,she will never have another shot at true love,643,646,she will never have another shot at true love,5ndJuQUhkQc,https://www.youtube.com/watch?v=5ndJuQUhkQc&t=...
697,coming from south america was actually coming,1357,1360,silver coming from south america was actually ...,Ob0ZkMVVNT8,https://www.youtube.com/watch?v=Ob0ZkMVVNT8&t=...
698,well also share some more general information,111,115,well also share some more general information,ocWqnsBA_Tg,https://www.youtube.com/watch?v=ocWqnsBA_Tg&t=...


In [10]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,without having um as much structure or as much...,2093,2111,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0,https://www.youtube.com/watch?v=iJkfTPN8MS0&t=...
1,lets talk about your action plans now and i an...,1870,1886,lets talk about your action plans now and i an...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
2,here in as an example of hours of operation ri...,933,951,here in as an example of hours of operation ri...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
3,norman jones remember the kind of big picture ...,2320,2335,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
4,de rose we appreciate you being here im glad i...,2799,2809,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
...,...,...,...,...,...,...
695,their grandmother one time told them oh my god...,640,644,their grandmother one time told them oh my god...,f3utieQjPbg,https://www.youtube.com/watch?v=f3utieQjPbg&t=...
696,results from all three species better than any...,1097,1102,can compare results from all three species bet...,n23xGtfdeJU,https://www.youtube.com/watch?v=n23xGtfdeJU&t=...
697,name from patients where social security numbe...,3852,3855,select name from patients where social securit...,gKAedzBZh58,https://www.youtube.com/watch?v=gKAedzBZh58&t=...
698,weapons had just taken place when he finally p...,5643,5650,weapons had just taken place when he finally p...,wCfMtDgOP94,https://www.youtube.com/watch?v=wCfMtDgOP94&t=...


In [11]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,theresa anderson and because i only talked wit...,2089,2100,theresa anderson and because i only talked wit...,iZcte874ICY,https://www.youtube.com/watch?v=iZcte874ICY&t=...
1,dr michelle taylor good afternoon everyone its...,2165,2183,dr michelle taylor good afternoon everyone its...,rZEc_NCcorU,https://www.youtube.com/watch?v=rZEc_NCcorU&t=...
2,norman jones remember the kind of big picture ...,2320,2337,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
3,de rose we appreciate you being here im glad i...,2799,2810,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
4,when all the students parents old school toget...,3051,3067,portia polk when all the students parents old ...,6dxtMxcFJ3M,https://www.youtube.com/watch?v=6dxtMxcFJ3M&t=...
...,...,...,...,...,...,...
695,before eventually moving onto australia where ...,511,515,and bali before eventually moving onto austral...,mn7_KppKOrs,https://www.youtube.com/watch?v=mn7_KppKOrs&t=...
696,from our ip village partner global affairs tra...,273,278,from our ip village partner global affairs tra...,6E5uisJUBp0,https://www.youtube.com/watch?v=6E5uisJUBp0&t=...
697,all those ten people were heroes no difference...,3013,3019,all those ten people were heroes no difference...,6BjJF073jhc,https://www.youtube.com/watch?v=6BjJF073jhc&t=...
698,specialty pharmacy patient therapy management ...,67,71,specialty pharmacy patient therapy management ...,3r8C4ppfA2A,https://www.youtube.com/watch?v=3r8C4ppfA2A&t=67s


In [12]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,that this is not just getting any old attorney...,5513,5527,katie kersh that this is not just getting any ...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,thats something you dont have to worry about a...,2199,2213,lauren opgenorth thats something you dont have...,HWTd4jxCixk,https://www.youtube.com/watch?v=HWTd4jxCixk&t=...
2,so prepare relevant questions and also on the ...,1444,1462,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs,https://www.youtube.com/watch?v=qgBqBnEJFgs&t=...
3,james moore depending on the graduation someti...,3463,3484,james moore depending on the graduation someti...,9nBekn1eQtg,https://www.youtube.com/watch?v=9nBekn1eQtg&t=...
4,this type of research yeah i think thats still...,3706,3724,this type of research yeah i think thats still...,Anatl2PLRag,https://www.youtube.com/watch?v=Anatl2PLRag&t=...
...,...,...,...,...,...,...
695,looks like im gon na flip these chicken thighs...,519,526,looks like im gon na flip these chicken thighs...,5QbJ53IPZCc,https://www.youtube.com/watch?v=5QbJ53IPZCc&t=...
696,people who lived some two hundred years ago st...,1863,1867,people who lived some two hundred years ago st...,T5jeT88EYcQ,https://www.youtube.com/watch?v=T5jeT88EYcQ&t=...
697,notified whenever mr phone puts out an awesome...,66,70,notified whenever mr phone puts out an awesome...,L10pac5c5ao,https://www.youtube.com/watch?v=L10pac5c5ao&t=66s
698,really interesting science their other strateg...,1770,1777,really interesting science their other strateg...,APv7JQXv09c,https://www.youtube.com/watch?v=APv7JQXv09c&t=...


In [13]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,brian hoffman but look at how different the ou...,1756,1772,brian hoffman but look at how different the ou...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,chrissy foster were seeing this pop up in so m...,5444,5464,chrissy foster were seeing this pop up in so m...,SnAFp1hTNP8,https://www.youtube.com/watch?v=SnAFp1hTNP8&t=...
2,and that is something thats really important e...,485,503,jillian deas and that is something thats reall...,twJkEtTvvEo,https://www.youtube.com/watch?v=twJkEtTvvEo&t=...
3,david shoemaker responded back to her in a pri...,2541,2554,david shoemaker responded back to her in a pri...,PRG60EnKnIo,https://www.youtube.com/watch?v=PRG60EnKnIo&t=...
4,teresa bill well and ill just share that i rea...,2082,2102,teresa bill well and ill just share that i rea...,tffd1kmM3e4,https://www.youtube.com/watch?v=tffd1kmM3e4&t=...
...,...,...,...,...,...,...
695,stage name dev click close add an artifact sel...,376,386,stage name dev click close add an artifact sel...,_0ELkrXRIe4,https://www.youtube.com/watch?v=_0ELkrXRIe4&t=...
696,few duplicate posts um feedback hub doesnt cur...,2704,2710,few duplicate posts um feedback hub doesnt cur...,URiJrRjqJXQ,https://www.youtube.com/watch?v=URiJrRjqJXQ&t=...
697,really thinking about what need isnt being met...,626,632,really thinking about what need isnt being met...,dq6Ly5DgBq4,https://www.youtube.com/watch?v=dq6Ly5DgBq4&t=...
698,however suppose search engine results turn up ...,62,68,however suppose search engine results turn up ...,3XkGjVnXgzU,https://www.youtube.com/watch?v=3XkGjVnXgzU&t=62s


#### End Time Reassign

In [14]:
df_28["end_time"] = df_28["start_time"] + 1
df_200["end_time"] = df_200["start_time"] + 1
df_1000["end_time"] = df_1000["start_time"] + 1
df_5000["end_time"] = df_5000["start_time"] + 1
df_10000["end_time"] = df_10000["start_time"] + 1
df_20000["end_time"] = df_20000["start_time"] + 1
df_40000["end_time"] = df_40000["start_time"] + 1

In [15]:
out_path = "/home/kurubal/Downloads"

In [16]:
df_28.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# !!!!! End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) !!!!! 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [16]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

In [17]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [18]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [19]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [20]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [21]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [22]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [23]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,and this is what it was for me and it was,2038,2039,and this is what it was for me and it was great,mO0oqUQ5eHc,https://www.youtube.com/watch?v=mO0oqUQ5eHc&t=...
1,what do we have to do and not do it this,2007,2008,but ok we dont what do we have to do and not d...,7eUrrQRpz2w,https://www.youtube.com/watch?v=7eUrrQRpz2w&t=...
2,and what this will do is this will,2226,2227,and what this will do is this will,UNnFRj9_TNo,https://www.youtube.com/watch?v=UNnFRj9_TNo&t=...
3,and what this will do is this will,274,275,and what this will do is this will,fgMD2wvpvpk,https://www.youtube.com/watch?v=fgMD2wvpvpk&t=...
4,what it is that you do and do not,624,625,what it is that you do and do not like,Gyqu5AER7gg,https://www.youtube.com/watch?v=Gyqu5AER7gg&t=...
...,...,...,...,...,...,...
371,it will,639,640,it will,LfnrRPFhkuY,https://www.youtube.com/watch?v=LfnrRPFhkuY&t=...
372,no will,557,558,no will,CkqI8IW8MlU,https://www.youtube.com/watch?v=CkqI8IW8MlU&t=...
373,it will,314,315,it will,zxiHkW22UrA,https://www.youtube.com/watch?v=zxiHkW22UrA&t=...
374,he will,2905,2906,he will,1Qz6fafTIEM,https://www.youtube.com/watch?v=1Qz6fafTIEM&t=...


In [24]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,so do you think life will work in the right wa...,560,561,so do you think life will work in the right wa...,IXhNNNeYXnA,https://www.youtube.com/watch?v=IXhNNNeYXnA&t=...
1,what do you think about this i mean yeah to me...,538,539,what do you think about this i mean yeah to me...,_aBzOI9DWb4,https://www.youtube.com/watch?v=_aBzOI9DWb4&t=...
2,things that are out there you can do nothing a...,937,938,things that are out there you can do nothing a...,ldlflaQVAAE,https://www.youtube.com/watch?v=ldlflaQVAAE&t=...
3,the way is where this is why i really want to ...,2610,2611,the way is where this is why i really want to ...,MXGmKbjEIlw,https://www.youtube.com/watch?v=MXGmKbjEIlw&t=...
4,say oh yeah yes she can and she would like to ...,1637,1638,say oh yeah yes she can and she would like to ...,dYa5scWadpc,https://www.youtube.com/watch?v=dYa5scWadpc&t=...
...,...,...,...,...,...,...
695,too much time before can help,10636,10637,too much time before can help,Yf7Vs6IWlLk,https://www.youtube.com/watch?v=Yf7Vs6IWlLk&t=...
696,really know who god was still,252,253,so i didnt really know who god was still,T_seNRh76aU,https://www.youtube.com/watch?v=T_seNRh76aU&t=...
697,anything going on or anything,1487,1488,to hear anything going on or anything,WqKV2veBRlc,https://www.youtube.com/watch?v=WqKV2veBRlc&t=...
698,never be good at anything now,127,128,ill never be good at anything now,WdnoMJ9KyX4,https://www.youtube.com/watch?v=WdnoMJ9KyX4&t=...


In [25]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,now this shot could have been the top shot of ...,199,200,now this shot could have been the top shot of ...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
1,so you can imagine if you go through all of th...,293,294,so you can imagine if you go through all of th...,K1GhQ8633Bg,https://www.youtube.com/watch?v=K1GhQ8633Bg&t=...
2,and so one of the questions people say to me i...,971,972,craig robinson and so one of the questions peo...,G3w3f6M_w54,https://www.youtube.com/watch?v=G3w3f6M_w54&t=...
3,its a little family that you turn into and jus...,49,50,its a little family that you turn into and jus...,GymR27z8RHM,https://www.youtube.com/watch?v=GymR27z8RHM&t=49s
4,and i remember when it happened i was like oh ...,796,797,ayanna howard and i remember when it happened ...,M_IRnJZ2xQ0,https://www.youtube.com/watch?v=M_IRnJZ2xQ0&t=...
...,...,...,...,...,...,...
695,because he was missing out on many fun things,453,454,because he was missing out on many fun things,L5jE2X3NZaM,https://www.youtube.com/watch?v=L5jE2X3NZaM&t=...
696,she will never have another shot at true love,643,644,she will never have another shot at true love,5ndJuQUhkQc,https://www.youtube.com/watch?v=5ndJuQUhkQc&t=...
697,coming from south america was actually coming,1357,1358,silver coming from south america was actually ...,Ob0ZkMVVNT8,https://www.youtube.com/watch?v=Ob0ZkMVVNT8&t=...
698,well also share some more general information,111,112,well also share some more general information,ocWqnsBA_Tg,https://www.youtube.com/watch?v=ocWqnsBA_Tg&t=...


In [26]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,without having um as much structure or as much...,2093,2094,sydnee yu give myself like the grace to explor...,iJkfTPN8MS0,https://www.youtube.com/watch?v=iJkfTPN8MS0&t=...
1,lets talk about your action plans now and i an...,1870,1871,lets talk about your action plans now and i an...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
2,here in as an example of hours of operation ri...,933,934,here in as an example of hours of operation ri...,nDOjm1UKsNM,https://www.youtube.com/watch?v=nDOjm1UKsNM&t=...
3,norman jones remember the kind of big picture ...,2320,2321,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
4,de rose we appreciate you being here im glad i...,2799,2800,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
...,...,...,...,...,...,...
695,their grandmother one time told them oh my god...,640,641,their grandmother one time told them oh my god...,f3utieQjPbg,https://www.youtube.com/watch?v=f3utieQjPbg&t=...
696,results from all three species better than any...,1097,1098,can compare results from all three species bet...,n23xGtfdeJU,https://www.youtube.com/watch?v=n23xGtfdeJU&t=...
697,name from patients where social security numbe...,3852,3853,select name from patients where social securit...,gKAedzBZh58,https://www.youtube.com/watch?v=gKAedzBZh58&t=...
698,weapons had just taken place when he finally p...,5643,5644,weapons had just taken place when he finally p...,wCfMtDgOP94,https://www.youtube.com/watch?v=wCfMtDgOP94&t=...


In [27]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,theresa anderson and because i only talked wit...,2089,2090,theresa anderson and because i only talked wit...,iZcte874ICY,https://www.youtube.com/watch?v=iZcte874ICY&t=...
1,dr michelle taylor good afternoon everyone its...,2165,2166,dr michelle taylor good afternoon everyone its...,rZEc_NCcorU,https://www.youtube.com/watch?v=rZEc_NCcorU&t=...
2,norman jones remember the kind of big picture ...,2320,2321,norman jones remember the kind of big picture ...,b9pudZegYgQ,https://www.youtube.com/watch?v=b9pudZegYgQ&t=...
3,de rose we appreciate you being here im glad i...,2799,2800,eugene gino de rose we appreciate you being he...,pNDXsUrO3Rs,https://www.youtube.com/watch?v=pNDXsUrO3Rs&t=...
4,when all the students parents old school toget...,3051,3052,portia polk when all the students parents old ...,6dxtMxcFJ3M,https://www.youtube.com/watch?v=6dxtMxcFJ3M&t=...
...,...,...,...,...,...,...
695,before eventually moving onto australia where ...,511,512,and bali before eventually moving onto austral...,mn7_KppKOrs,https://www.youtube.com/watch?v=mn7_KppKOrs&t=...
696,from our ip village partner global affairs tra...,273,274,from our ip village partner global affairs tra...,6E5uisJUBp0,https://www.youtube.com/watch?v=6E5uisJUBp0&t=...
697,all those ten people were heroes no difference...,3013,3014,all those ten people were heroes no difference...,6BjJF073jhc,https://www.youtube.com/watch?v=6BjJF073jhc&t=...
698,specialty pharmacy patient therapy management ...,67,68,specialty pharmacy patient therapy management ...,3r8C4ppfA2A,https://www.youtube.com/watch?v=3r8C4ppfA2A&t=67s


In [28]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,that this is not just getting any old attorney...,5513,5514,katie kersh that this is not just getting any ...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,thats something you dont have to worry about a...,2199,2200,lauren opgenorth thats something you dont have...,HWTd4jxCixk,https://www.youtube.com/watch?v=HWTd4jxCixk&t=...
2,so prepare relevant questions and also on the ...,1444,1445,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs,https://www.youtube.com/watch?v=qgBqBnEJFgs&t=...
3,james moore depending on the graduation someti...,3463,3464,james moore depending on the graduation someti...,9nBekn1eQtg,https://www.youtube.com/watch?v=9nBekn1eQtg&t=...
4,this type of research yeah i think thats still...,3706,3707,this type of research yeah i think thats still...,Anatl2PLRag,https://www.youtube.com/watch?v=Anatl2PLRag&t=...
...,...,...,...,...,...,...
695,looks like im gon na flip these chicken thighs...,519,520,looks like im gon na flip these chicken thighs...,5QbJ53IPZCc,https://www.youtube.com/watch?v=5QbJ53IPZCc&t=...
696,people who lived some two hundred years ago st...,1863,1864,people who lived some two hundred years ago st...,T5jeT88EYcQ,https://www.youtube.com/watch?v=T5jeT88EYcQ&t=...
697,notified whenever mr phone puts out an awesome...,66,67,notified whenever mr phone puts out an awesome...,L10pac5c5ao,https://www.youtube.com/watch?v=L10pac5c5ao&t=66s
698,really interesting science their other strateg...,1770,1771,really interesting science their other strateg...,APv7JQXv09c,https://www.youtube.com/watch?v=APv7JQXv09c&t=...


In [29]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,brian hoffman but look at how different the ou...,1756,1757,brian hoffman but look at how different the ou...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,chrissy foster were seeing this pop up in so m...,5444,5445,chrissy foster were seeing this pop up in so m...,SnAFp1hTNP8,https://www.youtube.com/watch?v=SnAFp1hTNP8&t=...
2,and that is something thats really important e...,485,486,jillian deas and that is something thats reall...,twJkEtTvvEo,https://www.youtube.com/watch?v=twJkEtTvvEo&t=...
3,david shoemaker responded back to her in a pri...,2541,2542,david shoemaker responded back to her in a pri...,PRG60EnKnIo,https://www.youtube.com/watch?v=PRG60EnKnIo&t=...
4,teresa bill well and ill just share that i rea...,2082,2083,teresa bill well and ill just share that i rea...,tffd1kmM3e4,https://www.youtube.com/watch?v=tffd1kmM3e4&t=...
...,...,...,...,...,...,...
695,stage name dev click close add an artifact sel...,376,377,stage name dev click close add an artifact sel...,_0ELkrXRIe4,https://www.youtube.com/watch?v=_0ELkrXRIe4&t=...
696,few duplicate posts um feedback hub doesnt cur...,2704,2705,few duplicate posts um feedback hub doesnt cur...,URiJrRjqJXQ,https://www.youtube.com/watch?v=URiJrRjqJXQ&t=...
697,really thinking about what need isnt being met...,626,627,really thinking about what need isnt being met...,dq6Ly5DgBq4,https://www.youtube.com/watch?v=dq6Ly5DgBq4&t=...
698,however suppose search engine results turn up ...,62,63,however suppose search engine results turn up ...,3XkGjVnXgzU,https://www.youtube.com/watch?v=3XkGjVnXgzU&t=62s


#### Copy Move And Delete

In [30]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

['English_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'English_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [31]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [32]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass